In [1]:
from sklearn import svm, datasets
from sklearn.model_selection import GridSearchCV
import os
import mlflow
import mlflow.sklearn
from  mlflow.tracking import MlflowClient
import pandas as pd
from tqdm.notebook import tqdm

In [2]:
tracking_uri = os.environ.get("TRACKING_URL")
client = MlflowClient(tracking_uri=tracking_uri)
mlflow.set_tracking_uri(tracking_uri)
experiments = client.list_experiments()
experiment_names = []
for exp in experiments:
    experiment_names.append(exp.name)
experiment_name = "sklearn_gridsearch"
if experiment_name not in experiment_names:
    mlflow.create_experiment(experiment_name)
mlflow.set_experiment(experiment_name)

In [3]:
iris = datasets.load_iris()
parameters = {'kernel':('linear', 'rbf'), 'C':[x/10 for x in range(1,10)]}
svc = svm.SVC()
clf = GridSearchCV(svc, parameters)
clf.fit(iris.data, iris.target)

GridSearchCV(estimator=SVC(),
             param_grid={'C': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
                         'kernel': ('linear', 'rbf')})

In [4]:
df = pd.DataFrame.from_dict(clf.cv_results_)

In [5]:
metrics = ['mean_fit_time','std_fit_time','mean_score_time','std_score_time','mean_test_score','std_test_score']
params  = ['param_C','param_kernel']
for _index in tqdm(df.index):
    mlflow_params  = df[params].loc[_index].to_dict()
    mlflow_metrics = df[metrics].loc[_index].to_dict()
    with mlflow.start_run():
        mlflow.log_params(mlflow_params)
        mlflow.log_metrics(mlflow_metrics)

In [6]:
[x/10 for x in range(1,10)]

[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]